In [1]:
import pandas as pd
from datetime import datetime
from collections import namedtuple
from util import *

In [2]:
def create_seq_dataset(year, month, maxlen=4):
    """
    year:현재년도
    month:현재월
    maxlen:시계열에서 고려하는 개월 수 
    ex) 현재 개월수가 11월이면, 11월/10월/9월/8월을 고려함
    """
    datelist = list_dates(year, month, maxlen=maxlen)
    dataset = load_dataset(datelist)
    return dataset
    

def load_dataset(datelist):
    data = []
    for date in datelist:
        data.append(load_file(date.year, date.month))
    print('data is loaded')
    return data
    
def list_dates(year, month, maxlen=4):
    mydate = namedtuple('mydate', ['year', 'month'])
    curdate = mydate(year, month)
    datelist = [curdate]
    for i in range(1, maxlen):
        year = curdate.year
        month = curdate.month - i
        if year == 2021 and month <= 0:
            year = 2020
            month += 12
        elif year == 2020 and month <= 0:
            return datelist       
        datelist.append(mydate(year, month))
    return datelist
            
    

In [3]:
create_seq_dataset(2021, 2, maxlen=4)

data/applog/applog_202102.csv
data/applog/applog_202102.csv is read
data/member/mbr_202102.csv


MemoryError: Unable to allocate 365. MiB for an array with shape (47871504,) and data type int64